# Anecdotal evidence of pyoframe's performance

The example brought up here is related to the unit-commitment problem for power grids, which is a mixed-integer linear programming problem. The solution to the model provides the most economical way of dispatching generators in a power grid.

For this demonstration of pyoframe's capabilities, we focus on a particular detail in setting up the unit-commitment related to the definition of where in the grid generation resources inject their power output.

The figure below illustrates the concept for a simple case, where there are four generation resources ("Res") and three defined nodes where power can be injected ("Gen"). Three Resources inject all their power at one single Generator, but Resource 4 divides its power output between Generator 2 and Generator 3. 

For the demonstration we use real-world data from the Texas power grid (ERCOT), which defines 787 relations between thermal generation resources and generators.

The demonstration contrasts pyoframe with gurobipy and [linopy](https://linopy.readthedocs.io/en/latest/), an opimization library based on [xarray](https://github.com/pydata/xarray). 

<img src="./three-bus-four-gen.png" alt="Three-bus grid" width="500">

In [11]:
%pip install linopy pyoframe pandas gurobipy fastparquet

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: linopy in c:\users\staadeck\software\pyoframe\.venv\lib\site-packages (0.5.3)




[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import gurobipy as gp
import linopy as lp
import pandas as pd
import polars as pl
from gurobipy import GRB

import pyoframe as pf

## Load data

In [13]:
py = (
    pl.read_parquet("./data_py.parquet").select("p", "y").unique()
)  # Set of thermal generation resources
# p is the index over individual plants, and y an index over possible modes (for combined-cycle plants).

t = pl.DataFrame({"t": range(34)})  # Time vector

# Indexes over which variables are defined.
pyt = py.join(t, how="cross")
pyt_tl = [tuple(row) for row in pyt.iter_rows()]
p_coords, y_coords, t_coords = (
    pd.Index(pyt["p"].unique()),
    pd.Index(pyt["y"].unique()),
    pd.Index(t["t"]),
)

In [14]:
pyt

p,y,t
i64,i64,i64
143,1,0
143,1,1
143,1,2
143,1,3
143,1,4
…,…,…
256,1,29
256,1,30
256,1,31


In [15]:
gen_py = pd.read_parquet("./gen_py.parquet").set_index(
    ["p", "y", "gen"]
)  # Relationship between resources and generators
gen_py_pl = pl.from_pandas(gen_py.reset_index())

## Instantiate models

In [16]:
gp_model = gp.Model("Example")
pf_model = pf.Model(sense="min")
lp_model = lp.Model()

Set parameter Username
Set parameter LicenseID to value 2596799
Academic license - for non-commercial use only - expires 2025-12-07


## Create variables

In [17]:
P_py_gp = gp_model.addVars(pyt_tl, vtype=GRB.CONTINUOUS, name="P_py")
gp_model.update()
P_py_lp = lp_model.add_variables(
    coords=[p_coords, y_coords, t_coords],
    lower=0.0,
    name="P_py",
)
pf_model.P_py = pf.Variable(pyt, lb=0)

## Memory use

### Amount of memory occupied by linopy variable

In [18]:
memory_usage_lp = sum(var.nbytes for var in P_py_lp.data.variables.values())
memory_usage_MB_lp = memory_usage_lp / (1024**2)

print(f"Memory used by P_py_lp: {memory_usage_MB_lp:.2f} MB")

Memory used by P_py_lp: 5.71 MB


### Amount of memory occupied by pyoframe variable

In [19]:
memory_usage_pf = pf_model.P_py.data.estimated_size()
memory_usage_MB_pf = memory_usage_pf / (1024**2)

print(f"Memory used by P_py (pyoframe): {memory_usage_MB_pf:.2f} MB")

Memory used by P_py (pyoframe): 0.45 MB


### Pyoframe is **leaner**!

In [20]:
# Improvement
print(
    f"Pyoframe variable uses {memory_usage_MB_pf / memory_usage_MB_lp * 100: .1f} % of memory of linopy variable"
)

Pyoframe variable uses  7.8 % of memory of linopy variable


The reason for this is that pyoframe stores variables in apolars dataframe, in what is essentially a coordinate format (coo) sparse representation. Linopy uses xarray, which is a dense representation over the indexes p, y and t.

## Speed

### Mapping generator output to busses in Gurobipy

In [21]:
def gp_generation_at_each_node(P_py, gen_py, time_periods):
    return gp.tupledict(
        (
            (gen, t_),
            gp.quicksum(
                gen_py.loc[p, y, gen] * P_py[(p, y, t_)]
                for p, y in gen_py.xs(key=gen, level="gen").index
                if (p, y, t_) in P_py
            ),
        )
        for gen in gen_py.index.get_level_values("gen")
        for t_ in time_periods
    )  # The power generated at each gen due to generation at the thermal plants

In [25]:
result_gp = %timeit -o -n 2 -r 1 gp_generation_at_each_node(P_py_gp, gen_py["portion"], t["t"])

14.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


In [26]:
P_gen_gp = gp_generation_at_each_node(P_py_gp, gen_py["portion"], t["t"])

In [27]:
P_gen_gp[(669, 1)]

<gurobi.LinExpr: 0.337254902 P_py[348,5,1]>

### Mapping generator output to busses in pyoframe

In [28]:
pf_model.P_gen = pf.sum(["p", "y"], pf_model.P_py * gen_py_pl)

In [29]:
pf_model.P_gen.filter(gen=669, t=1)

<Expression size=1 dimensions={'t': 1, 'gen': 1} terms=1>
[1,669]: 0.33725 P_py[348,5,1]

In [30]:
result_pf = %timeit -o  -n 4 -r 4  pf.sum(["p", "y"], pf_model.P_py * gen_py_pl)

18.6 ms ± 10.5 ms per loop (mean ± std. dev. of 4 runs, 4 loops each)


### Pyoframe is **faster**

In [31]:
print(f"Pyoframe is {result_gp.best / result_pf.best: .1f} times faster than gurobipy")

Pyoframe is  1239.1 times faster than gurobipy


The reason for this improvement of three  orders of magnitude is that the gurobipy version with tupledict requires a triple loop in python to formulate the relationship, whereas the operation in pyoframe is a join operation on polars dataframes. 

## Readability
Readability is of course hard to measure. This is just an example, and the reader will have to judge for herself. In our humble opinion, it is easier to see what is going on in the pyoframe version.

### GAMS
```gams
 eq_P_gen(gen,t)..  
    P_gen(gen,t) =e= sum(p, sum(y$(py(p,y,"py") and (gen_py(p,y,gen,"portion") gt 0)), 
                                 P_py(p,y,t)*gen_py(p,y,gen,"portion")
                                )
                     )
```

### PyoFrame
```python
self.P_gen_py = pf.sum(["p", "y"], self.model.P_py * gen_py)
```
and further below
```python
self.model.eq_P_gen = (
    self.generators().keep_extras()
    - self.P_gen_py.keep_extras()
    - self.P_gen_hx.keep_extras()
    - self.P_gen_v.keep_extras()
    - self.P_gen_bat.keep_extras()
    == 0
)
```